In [1]:
import os
import numpy as np
import pandas as pd

# from tqdm import tqdm 
# tqdm.pandas(desc="progress-bar")

import gensim 
import string
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

LabeledSentence = gensim.models.doc2vec.LabeledSentence

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#os.chdir('C:\\Users\\BTHANISH\\Documents\\Thanish\\Competition\\Hacker earth\\Amazon Hiring Challenge')

import xgboost as xgb

In [2]:
import tensorflow as tf
import tensorflow_hub as hub


W0810 06:05:19.526812 140651333011264 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
DF_train_prod = pd.read_csv('train.csv')
DF_test_prod = pd.read_csv('test.csv')
#DF_test_prod = DF_test_prod.loc[~DF_test_prod.duplicated(),]

print(DF_train_prod.shape, DF_test_prod.shape)

DF_test_prod['topic'] = 'Test_dataset'
DF_prod = pd.concat([DF_train_prod, DF_test_prod]).reset_index(drop = True)
DF_prod.columns = ['Review_text', 'Review_Title', 'topic']
DF_prod.head()

(5959, 3) (2553, 2)


,Review_text,Review_Title,topic
0,"Did nothing for me, didn't help lost even with...",Useless,Shipment and delivery
1,"Did nothing for me, didn't help lost even with...",Useless,Not Effective
2,I have bought these bags and immediately open...,TRASH!!! Do not buy these bags it’s a waist of...,Customer Service
3,Gave me an allergic reaction on my face :(,Do not recommend,Allergic
4,These don't compare to the name brand wipes. F...,Can't tackle big messes,Texture


In [4]:
# nltk.set_proxy('https://19.12.1.40:83')
# nltk.download(['punkt', 'stopwords'])

In [5]:
stop = stopwords.words('english')

# Tokenizing the two columns Review_text, Review_Title
for col in ['Review_text', 'Review_Title']:
    #Converting the columns to lowercase
    DF_prod[col] = DF_prod[col].apply(lambda x :  x.lower())

    #Replacing the puncutations
    DF_prod[col] = DF_prod[col].str.replace('[^\w\s]','')
    
    # Removing stop words
    DF_prod[col] = DF_prod[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    # Remove numerics from the data
    DF_prod[col] = DF_prod[col].str.replace('\d+', '')
    
    #Getting the tokens for the two columns
    new_col = col + '_tokens'
    DF_prod[new_col] = DF_prod.apply(lambda row : word_tokenize(row[col]), axis = 1)

# Combined tokens
DF_prod['combined_tokens'] = DF_prod['Review_text_tokens'] + DF_prod['Review_Title_tokens']
DF_prod.head(10)

,Review_text,Review_Title,topic,Review_text_tokens,Review_Title_tokens,combined_tokens
0,nothing didnt help lost even working eating he...,useless,Shipment and delivery,"[nothing, didnt, help, lost, even, working, ea...",[useless],"[nothing, didnt, help, lost, even, working, ea..."
1,nothing didnt help lost even working eating he...,useless,Not Effective,"[nothing, didnt, help, lost, even, working, ea...",[useless],"[nothing, didnt, help, lost, even, working, ea..."
2,bought bags immediately open one put trash bag...,trash buy bags waist time,Customer Service,"[bought, bags, immediately, open, one, put, tr...","[trash, buy, bags, waist, time]","[bought, bags, immediately, open, one, put, tr..."
3,gave allergic reaction face,recommend,Allergic,"[gave, allergic, reaction, face]",[recommend],"[gave, allergic, reaction, face, recommend]"
4,dont compare name brand wipes family little k...,cant tackle big messes,Texture,"[dont, compare, name, brand, wipes, family, li...","[cant, tackle, big, messes]","[dont, compare, name, brand, wipes, family, li..."
5,dont compare name brand wipes family little k...,cant tackle big messes,Quality/Contaminated,"[dont, compare, name, brand, wipes, family, li...","[cant, tackle, big, messes]","[dont, compare, name, brand, wipes, family, li..."
6,dont compare name brand wipes family little k...,cant tackle big messes,Color and texture,"[dont, compare, name, brand, wipes, family, li...","[cant, tackle, big, messes]","[dont, compare, name, brand, wipes, family, li..."
7,good,tastes horrible,Bad Taste/Flavor,[good],"[tastes, horrible]","[good, tastes, horrible]"
8,extremely hard swallow pills huge sides sharp ...,choking hazard,Too big to swallow,"[extremely, hard, swallow, pills, huge, sides,...","[choking, hazard]","[extremely, hard, swallow, pills, huge, sides,..."
9,first style leaving review solimo incontinent ...,bring back old style,Quality/Contaminated,"[first, style, leaving, review, solimo, incont...","[bring, back, old, style]","[first, style, leaving, review, solimo, incont..."


# Splitting the data back to train prod and test prod

In [6]:
train_prod = DF_prod.loc[DF_prod.topic != 'Test_dataset',].reset_index(drop = True)
test_prod = DF_prod.loc[DF_prod.topic == 'Test_dataset',].reset_index(drop = True)

print(train_prod.shape, test_prod.shape)

# Get the indep and dep features
indep = ['combined_tokens']
dep = ['topic']

(5959, 6) (2553, 6)


In [7]:
# Convert the labels to integer
LE = LabelEncoder()
LE.fit(train_prod.topic)
train_prod.topic = LE.transform(train_prod.topic)


# ELMO

In [8]:
# # Download the elmo model for the first time using proxy
# module_url = "https://tfhub.dev/google/elmo/2"
# print("Loading model from {}".format(module_url))
# os.environ["https_proxy"] = "http://19.12.1.40:83"
# elmo = hub.Module(module_url, trainable=True)

In [14]:
#Call the model from the next time
elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)

# Training on sentenes

In [10]:
# Training on sentenes
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
    
    #return embeddings
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [12]:
# Train on sentence

# threshold = 100
# list_train = [train_prod[i:i+threshold] for i in range(0, train_prod.shape[0], threshold)]
# list_test  = [test_prod[i:i+threshold] for i in range(0, test_prod.shape[0], threshold)]

# # Extract ELMo embeddings
# elmo_train = [elmo_vectors(x['Review_text']) for x in list_train]
# elmo_test = [elmo_vectors(x['Review_text']) for x in list_test]

# elmo_train_prod = np.concatenate([elmo_train], axis = 0)
# elmo_test_prod = np.concatenate([elmo_test], axis = 0)

In [13]:
# # save elmo_train_new
# pickle_out = open("elmo_train_10082019.pickle","wb")
# pickle.dump(elmo_train_prod, pickle_out)
# pickle_out.close()

# # save elmo_test_new
# pickle_out = open("elmo_test_10082019.pickle","wb")
# pickle.dump(elmo_test_prod, pickle_out)
# pickle_out.close()

# Training on tokens

In [33]:
def elmo_vectors(x):
    token_input = x
    tokens_length = [x.shape[0], max_length_sequence]    
    embeddings = elmo(inputs = {"tokens": tokens_input ,"sequence_len": tokens_length}, 
                      signature="tokens", as_dict=True)["elmo"]
    
    #return embeddings
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))


In [ ]:
# Train on tokens
threshold = 100
list_train = [train_prod[i:i+threshold] for i in range(0, train_prod.shape[0], threshold)]
list_test  = [test_prod[i:i+threshold] for i in range(0, test_prod.shape[0], threshold)]

# Extract ELMo embeddings
max_length_sequence = train_prod['combined_tokens'].apply(len).max() #max length of the sentence in the corpus i.e.(maximum word in a sentence)

elmo_train = [elmo_vectors(x['combined_tokens']) for x in list_train]
elmo_test = [elmo_vectors(x['combined_tokens']) for x in list_test]

elmo_train_prod = np.concatenate([elmo_train], axis = 0)
elmo_test_prod = np.concatenate([elmo_test], axis = 0)

print("Train shape",elmo_train_prod.shape, "Test shape", elmo_test_prod.shape)

In [36]:
# # save elmo_train_new
# pickle_out = open("elmo_train_tokens_10082019.pickle","wb")
# pickle.dump(elmo_train_prod, pickle_out)
# pickle_out.close()

# # save elmo_test_new
# pickle_out = open("elmo_test_tokens_10082019.pickle","wb")
# pickle.dump(elmo_test_prod, pickle_out)
# pickle_out.close()

# Loading elmo models

In [43]:
pickle_in = open("elmo_train_10082019.pickle", "rb")
#pickle_in = open("elmo_train_tokens_10082019.pickle", "rb") # if you need tokens
elmo_train_prod_DF = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_10082019.pickle", "rb")
#pickle_in = open("elmo_test_tokens_10082019.pickle", "rb") # if you need tokens
elmo_test_prod_DF = pickle.load(pickle_in)

elmo_train_prod_DF.shape, elmo_test_prod_DF.shape

((60,), (26,))

In [44]:
elmo_train_prod_DF[0]

array([[-0.00412273, -0.05918864,  0.06556313, ...,  0.01515634,
         0.01788805, -0.0090268 ],
       [-0.00412273, -0.05918864,  0.06556313, ...,  0.01515634,
         0.01788805, -0.0090268 ],
       [ 0.01752855,  0.01659176,  0.07129052, ...,  0.03331851,
        -0.07741871,  0.04769161],
       ...,
       [-0.00112581, -0.03386815,  0.03087963, ...,  0.0358163 ,
        -0.01794741,  0.01692263],
       [-0.00112581, -0.03386815,  0.03087963, ...,  0.0358163 ,
        -0.01794741,  0.01692263],
       [-0.00112581, -0.03386815,  0.03087963, ...,  0.0358163 ,
        -0.01794741,  0.01692263]], dtype=float32)

In [45]:
#Reshaping the arrays 

def reshaping_DF (DF):
    a = np.empty([0,1024], )
    for i in range(DF.shape[0]):
    #     print(elmo_train_prod[lisi].shape)
    #     print('Head')
    #     print(elmo_train_prod[lisi][:3])
    #     print('tail')
    #     print(elmo_train_prod[lisi][-3:])

    #     print("")
        a = np.concatenate((a, DF[i]), axis=0)
    #print(a.shape)
    return a


elmo_train_prod = reshaping_DF(elmo_train_prod_DF)
elmo_test_prod = reshaping_DF(elmo_test_prod_DF)
elmo_train_prod.shape, elmo_test_prod.shape

((5959, 1024), (2553, 1024))

# Split train and test

In [62]:
train_stop = int(train_prod.shape[0]*0.7)

np.random.seed(100)
# train_local_X, test_local_X, train_local_Y, test_local_Y = train_test_split(train_prod[indep],
#                                                                             train_prod[dep],
#                                                                             test_size = 0.2)
train_local_X = elmo_train_prod[0 : train_stop, 0:750]
train_local_Y = train_prod[dep][0 : train_stop]
test_local_X = elmo_train_prod[train_stop : train_prod.shape[0], 0:750]
test_local_Y = train_prod[dep][train_stop : train_prod.shape[0]]

train_prod_X = elmo_train_prod
train_prod_Y = train_prod[dep]
test_prod_X = elmo_test_prod

print(train_local_X.shape, train_local_Y.shape, test_local_X.shape, test_local_Y.shape, train_prod_X.shape, test_prod_X.shape)

(4171, 750) (4171, 1) (1788, 750) (1788, 1) (5959, 1024) (2553, 1024)


# RandomForest 

#### Training on Local 

In [47]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators = 100)
RF.fit(train_local_X, train_local_Y)
RF_local_pred = RF.predict(test_local_X)

print(sum(RF_local_pred == test_local_Y.topic)/len(test_local_Y.topic))
(RF_local_pred == test_local_Y.topic).value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.39988814317673377


False    1073
True      715
Name: topic, dtype: int64

In [48]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators = 110)
RF.fit(train_local_X, train_local_Y)
RF_local_pred = RF.predict(test_local_X)
print(sum(RF_local_pred == test_local_Y.topic)/len(test_local_Y.topic))
(RF_local_pred == test_local_Y.topic).value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.40548098434004476


False    1063
True      725
Name: topic, dtype: int64

#### on prod

In [ ]:
# # test prod
# RF_sub = DF_test_prod.copy()
# RF_prod_pred = RF.predict(test_prod_vecs_w2v)
# RF_sub['topic'] = LE.inverse_transform(RF_prod_pred)
# RF_sub.to_csv('RF_sub_4.csv', index = False)
# #RF_sub.tail(10)


#### RF Test local

In [55]:
#test local
RF_local_pred = RF.predict_proba(test_local_X)




train_local_X = train_prod.loc[0 : train_stop, ]
train_local_Y = train_prod[dep][0 : train_stop]
test_local_X = train_prod.loc[train_stop : train_prod.shape[0], ]
test_local_Y = train_prod[dep][train_stop : train_prod.shape[0]]

train_prod_X = elmo_train_prod
train_prod_Y = train_prod[dep]
test_prod_X = elmo_test_prod





test_local_DF = pd.DataFrame({'tokens': test_local_X.combined_tokens,
                              'Actual' : LE.inverse_transform(test_local_Y.topic)
                             })
test_local_DF.tokens = test_local_DF.tokens.astype('str')

top_n_predictions = np.fliplr(np.argsort(RF_local_pred, axis = 1))
#top_class = LE.inverse_transform(RF.classes_[top_n_predictions])
top_class = np.array([LE.inverse_transform(i) for i in RF.classes_[top_n_predictions]])

test_local_DF['predicted'] = top_class.tolist()

temp = test_local_DF.groupby(['tokens'])['Actual'].count().reset_index(drop = False).rename(columns = {'Actual':'seq'})
#print(temp.head())

#print(test_local.shape)
test_local_DF = test_local_DF.merge(temp, how = 'left', 
                              left_on = ['tokens'], right_on = ['tokens'])
#print(test_local.shape)

final_list = []
test_local_DF['predicted_trim'] = None
for index, row in test_local_DF.iterrows():
    temp = test_local_DF['predicted'][index][0: (test_local_DF['seq'][index])]
    final_list.append(temp)
test_local_DF['predicted_trim'] = final_list
    

final_list = []
for index, row in test_local_DF.iterrows():
    temp = np.isin(test_local_DF.Actual[index], (test_local_DF['predicted_trim'][index])).astype('int')
    final_list.append(temp)
test_local_DF['predicted_int'] = final_list
    
accuracy = sum(test_local_DF.predicted_int) / len(test_local_DF.predicted_int)
print(test_local_DF.predicted_int.value_counts())
print("Accuracy :",accuracy)

test_local_DF.head()

1    920
0    868
Name: predicted_int, dtype: int64
Accuracy : 0.5145413870246085


,tokens,Actual,predicted,seq,predicted_trim,predicted_int
0,"['ive', 'received', 'minutes', 'ago', 'opened'...",Shipment and delivery,"[Packaging, Shipment and delivery, Customer Se...",2,"[Packaging, Shipment and delivery]",1
1,"['ive', 'received', 'minutes', 'ago', 'opened'...",Packaging,"[Packaging, Shipment and delivery, Customer Se...",2,"[Packaging, Shipment and delivery]",1
2,"['disappointed', 'wasted', 'terrible', 'produc...",Smells Bad,"[Bad Taste/Flavor, Quality/Contaminated, Not E...",1,[Bad Taste/Flavor],0
3,"['broken', 'fibers', 'ratio', 'much', 'higher'...",Packaging,"[Not Effective, Allergic, Packaging, Bad Taste...",1,[Not Effective],0
4,"['pills', 'much', 'large', 'better', 'take', '...",Too big to swallow,"[Too big to swallow, Not Effective, Quality/Co...",1,[Too big to swallow],1


#### Training on prod 

In [ ]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators = 80)
RF.fit(train_prod_vecs_w2v, train_prod_Y)
RF_local_pred = RF.predict(test_local_vecs_w2v)
print(sum(RF_local_pred == test_local_Y.topic)/len(test_local_Y.topic))
(RF_local_pred == test_local_Y.topic).value_counts()

#### RF Test prod

In [ ]:
## test prod
RF_prod_pred = RF.predict_proba(test_local_X)
DF_test_prod = pd.read_csv('test.csv')

top_n_predictions = np.fliplr(np.argsort(RF_prod_pred, axis = 1))
top_class = LE.inverse_transform(RF.classes_[top_n_predictions])

DF_test_prod['topic'] = top_class.tolist()
DF_test_prod['topic_new'] = top_class.tolist()
DF_test_prod = DF_test_prod.loc[~DF_test_prod['Review Text'].duplicated(),]

DF_test_prod.head()

submission = pd.read_csv('Test.csv')
submission['seq'] = submission.groupby(['Review Text']).cumcount()

print(submission.shape, DF_test_prod.shape)
#submission.head()

submission = submission.merge(DF_test_prod[['Review Text', 'topic']], how = 'left',
                              left_on = ['Review Text'], right_on = ['Review Text'] )
submission.head(10)

final_list = []
for index, row in submission.iterrows():
#     print(index)
#     print(submission['seq'][index])
#     print("")
#     print(submission['topic'][index][submission['seq'][index]])
    temp = submission['topic'][index][submission['seq'][index]]
#   temp = submission['topic'][index][0]
    final_list.append(temp)
submission['topic'] = final_list
submission.drop(['seq'], axis =1, inplace = True)

submission.to_csv('RF_sub_7.csv', index = False)
submission.head()


# Gradient Boosting

#### training on local

In [ ]:
%%time

np.random.seed(100)
GBM = GradientBoostingClassifier(n_estimators=100)
GBM.fit(train_local_vecs_w2v, train_local_Y)
GBM_local_pred = GBM.predict(test_local_vecs_w2v)
print(sum(GBM_local_pred == test_local_Y.topic)/len(test_local_Y.topic))
(GBM_local_pred == test_local_Y.topic).value_counts()

In [ ]:
%%time

np.random.seed(100)
GBM = GradientBoostingClassifier(n_estimators=120)
GBM.fit(train_local_vecs_w2v, train_local_Y)
GBM_local_pred = GBM.predict(test_local_vecs_w2v)
print(sum(GBM_local_pred == test_local_Y.topic)/len(test_local_Y.topic))
(GBM_local_pred == test_local_Y.topic).value_counts()

In [ ]:
# GBM_sub = pd.read_csv('test.csv')
# GBM_prod_pred = GBM.predict(test_prod_vecs_w2v)
# GBM_sub['topic'] = LE.inverse_transform(GBM_prod_pred)
# GBM_sub.to_csv('GBM_sub_6.csv', index = False)
# GBM_sub.head(10)


#### GBM Test local

In [ ]:
GBM_local_pred = GBM.predict_proba(test_local_vecs_w2v)

test_local_DF = pd.DataFrame({'tokens': test_local_X.combined_tokens,
                              'Actual' : LE.inverse_transform(test_local_Y.topic)
                             })
test_local_DF.tokens = test_local_DF.tokens.astype('str')

top_n_predictions = np.fliplr(np.argsort(GBM_local_pred, axis = 1))
top_class = LE.inverse_transform(GBM.classes_[top_n_predictions])
test_local_DF['predicted'] = top_class.tolist()

temp = test_local_DF.groupby(['tokens'])['Actual'].count().reset_index(drop = False).rename(columns = {'Actual':'seq'})
#print(temp.head())

#print(test_local.shape)
test_local_DF = test_local_DF.merge(temp, how = 'left', 
                                    left_on = ['tokens'], right_on = ['tokens'])
#print(test_local.shape)

final_list = []
test_local_DF['predicted_trim'] = None
for index, row in test_local_DF.iterrows():
    temp = test_local_DF['predicted'][index][0: (test_local_DF['seq'][index])]
    final_list.append(temp)
test_local_DF['predicted_trim'] = final_list
    
final_list = []
for index, row in test_local_DF.iterrows():
    temp = np.isin(test_local_DF.Actual[index], (test_local_DF['predicted_trim'][index])).astype('int')
    final_list.append(temp)
test_local_DF['predicted_int'] = final_list
    
accuracy = sum(test_local_DF.predicted_int) / len(test_local_DF.predicted_int)
print(test_local_DF.predicted_int.value_counts())
print("Accuracy :",accuracy)

test_local_DF.head()

#### training on prod

In [ ]:
%%time

np.random.seed(100)
GBM = GradientBoostingClassifier(n_estimators=100)
GBM.fit(train_prod_vecs_w2v, train_prod_Y)
GBM_local_pred = GBM.predict(test_local_vecs_w2v)
print(sum(GBM_local_pred == test_local_Y.topic)/len(test_local_Y.topic))
(GBM_local_pred == test_local_Y.topic).value_counts()

#### GBM Test prod

In [ ]:
GBM_prod_pred = GBM.predict_proba(test_prod_vecs_w2v)
DF_test_prod = pd.read_csv('test.csv')

top_n_predictions = np.fliplr(np.argsort(GBM_prod_pred, axis = 1))
top_class = GBM.classes_[top_n_predictions]
DF_test_prod['topic'] = top_class.tolist()
DF_test_prod = DF_test_prod.loc[~DF_test_prod['Review Text'].duplicated(),]

DF_test_prod.head()

submission = pd.read_csv('Test.csv')
submission['seq'] = submission.groupby(['Review Text']).cumcount()

print(submission.shape, DF_test_prod.shape)
#submission.head()

submission = submission.merge(DF_test_prod[['Review Text', 'topic']], how = 'left',
                              left_on = ['Review Text'], right_on = ['Review Text'] )
submission.head(10)

final_list = []
for index, row in submission.iterrows():
#     print(index)
#     print(submission['seq'][index])
#     print("")
#     print(submission['topic'][index][submission['seq'][index]])
    temp = submission['topic'][index][submission['seq'][index]]
#   temp = submission['topic'][index][0]
    final_list.append(temp)
submission['topic'] = final_list
submission['topic'] = LE.inverse_transform(submission['topic'])
submission.drop(['seq'], axis =1, inplace = True)

submission.to_csv('GBM_sub_9.csv', index = False)
submission.head()


# XGBOOST

In [ ]:
train_local_X.shape, train_local_Y.topic.shape

In [33]:
dtrain_local = xgb.DMatrix(data = train_local_X, label = train_local_Y.topic)
dtest_local = xgb.DMatrix(data = test_local_X, label = test_local_Y.topic)
dtrain_prod = xgb.DMatrix(data = train_prod_X, label = train_prod_Y.topic)
dtest_prod = xgb.DMatrix(data = test_prod_X)


num_rounds = 1000

params = {'objective' : 'multi:softprob',
          'num_class' : len(train_local_Y.topic.unique()),
          #'eval_metric': 'auc',
          'max_depth' : 6,
          'eta' : 0.3,
          'subsample': 1,
          'colsample_bytree': 1,
          'silent' : 1
          ,'tree_method' : 'gpu_hist'
          }

eval_set = [(dtrain_local,'train'), (dtest_local,'test')]

In [ ]:
#Stopping. Best iteration:
#[106]	train-merror:0.288111	test-merror:0.488255
        
np.random.seed(100)
num_rounds = 1000
np.random.seed(100)
xgb_model = xgb.train(params,
                      dtrain_local,
                      evals = eval_set,
                      num_boost_round = num_rounds,
                      #feval = custom_mse,
                      verbose_eval = True
                      ,early_stopping_rounds = 30
                     )

xgb_local_pred = xgb_model.predict(dtest_local)

#Feature importance
pd.DataFrame.from_dict(xgb_model.get_score(), orient = 'index').rename(columns = {0:'importance'}).sort_values(['importance'], ascending = False)


In [37]:
np.random.seed(100)
xgb_model_prod = xgb.train(params,
                           dtrain_prod,
                           #evals = eval_set,
                           num_boost_round = xgb_model.best_iteration + 10
                           #feval = custom_mse,
                           verbose_eval = True,
                           #early_stopping_rounds = 20
                          )


xgb_prod_pred = xgb_model_prod.predict(dtest_prod)


In [50]:

top_n_predictions = np.fliplr(np.argsort(xgb_prod_pred, axis = 1))
#top_class = LE.inverse_transform(top_n_predictions)
# DF_test_prod['topic'] = top_class.tolist()
# DF_test_prod = DF_test_prod.loc[~DF_test_prod['Review Text'].duplicated(),]
top_n_predictions
LE.inverse_transform(top_n_predictions[0:1,0])

array(['Bad Taste/Flavor'], dtype=object)

In [ ]:
#XGB_prod_pred = xgb_model_prod.predict_proba(test_prod_vecs_w2v)
DF_test_prod = pd.read_csv('test.csv')

top_n_predictions = np.fliplr(np.argsort(xgb_prod_pred, axis = 1))
top_class = LE.inverse_transform(top_n_predictions)
DF_test_prod['topic'] = top_class.tolist()
DF_test_prod = DF_test_prod.loc[~DF_test_prod['Review Text'].duplicated(),]

DF_test_prod.head()

submission = pd.read_csv('Test.csv')
submission['seq'] = submission.groupby(['Review Text']).cumcount()

print(submission.shape, DF_test_prod.shape)
#submission.head()

submission = submission.merge(DF_test_prod[['Review Text', 'topic']], how = 'left',
                              left_on = ['Review Text'], right_on = ['Review Text'] )
submission.head(10)

final_list = []
for index, row in submission.iterrows():
#     print(index)
#     print(submission['seq'][index])
#     print("")
#     print(submission['topic'][index][submission['seq'][index]])
    temp = submission['topic'][index][submission['seq'][index]]
#   temp = submission['topic'][index][0]
    final_list.append(temp)
submission['topic'] = final_list
#submission['topic'] = LE.inverse_transform(submission['topic'])
submission.drop(['seq'], axis =1, inplace = True)

submission.to_csv('XGB_sub_9.csv', index = False)
submission.head()
